In [ ]:
pip install psycopg2
pip install psycopg2-binary

In [2]:
import os
os.chdir("../")

In [10]:
import pandas as pd
import psycopg2
import csv

from dataloading.postgresLoader import PostgresLoader

In [3]:
df = pd.read_csv('data/encounters.csv')
df.columns

Index(['Id', 'START', 'STOP', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER',
       'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION', 'BASE_ENCOUNTER_COST',
       'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'REASONCODE',
       'REASONDESCRIPTION'],
      dtype='object')

In [4]:
len(df.columns)

15

In [3]:
DB_NAME = os.environ.get("DB_NAME")
DB_USER = os.environ.get("DB_USER")
DB_PASSWORD = os.environ.get("DB_PASSWORD")
DB_HOST = os.environ.get("DB_HOST")
DB_PORT = os.environ.get("DB_PORT")

In [11]:
loader = PostgresLoader(db_name=DB_NAME, db_user=DB_USER, db_password=DB_PASSWORD)
loader.make_connection()

DB connection Established.


In [12]:
CSV_FILE_PATH = '/Users/rohituddagiri/Documents/Python Scripts/AI Projects/patient-journey-data-model/database/data/patients.csv'
TABLE_NAME = 'stg_patients'

# DB_COLUMNS = ["patientid", "memberid", "startdate", "enddate", "payerid", "planownership", "ownername"]

DB_COLUMNS = loader.get_columns(table_name=TABLE_NAME)


In [13]:
DB_COLUMNS

['patientid',
 'birthdate',
 'deathdate',
 'ssn',
 'drivers',
 'passport',
 'prefix',
 'firstname',
 'middlename',
 'lastname',
 'suffix',
 'maiden',
 'martial',
 'race',
 'ethinicity',
 'gender',
 'birthplace',
 'address',
 'city',
 'state',
 'county',
 'fips',
 'zip',
 'lat',
 'lon',
 'healthcareexpenses',
 'healthcarecoverage',
 'income']

In [14]:
loader.load_csv_to_table(csv_file=CSV_FILE_PATH,table_name=TABLE_NAME,db_cols=DB_COLUMNS)


 Processing /Users/rohituddagiri/Documents/Python Scripts/AI Projects/patient-journey-data-model/database/data/patients.csv --> stg_patients...
Error processing /Users/rohituddagiri/Documents/Python Scripts/AI Projects/patient-journey-data-model/database/data/patients.csv: unhashable type: 'list'


In [9]:
loader.close_connection()

DB Connection Closed.


In [17]:
insert_query = f"INSERT INTO {TABLE_NAME} ({', '.join(DB_COLUMNS)}) VALUES ({', '.join(['%s'] * len(DB_COLUMNS))})"
insert_query

'INSERT INTO stg_patients (patientid, birthdate, deathdate, ssn, drivers, passport, prefix, firstname, middlename, lastname, suffix, maiden, martial, race, ethinicity, gender, birthplace, address, city, state, county, fips, zip, lat, lon, healthcareexpenses, healthcarecoverage, income) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [15]:
with open(CSV_FILE_PATH, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    conn = psycopg2.connect(dbname=DB_NAME, 
                user=DB_USER,
                password=DB_PASSWORD,
                host=DB_HOST,
                port=DB_PORT)
    cur = conn.cursor()
    
    copy_sql = f"copy {TABLE_NAME} from stdin delimiter as ','"
    
    cur.copy_expert(copy_sql, f)
    
    # for row in reader:
    #     # selected_rows = tuple(row[col] for col in CSV_COLUMNS)
    #     # print([tuple(row[col] for col in row)])
    #     insert_data.append(tuple(row[col] for col in row))
    #     try:
    #         cur.executemany(insert_query, insert_data)
    #         conn.commit()
    #     except Exception as e:
    #         print(f"Skipping row due to error: {row} -> {e}")
    
conn.commit()
conn.close()
            

In [ ]:
conn = psycopg2.connect(dbname=DB_NAME, 
                user=DB_USER,
                password=DB_PASSWORD,
                host=DB_HOST,
                port=DB_PORT)
cur = conn.cursor()

cur.execute("""SELECT column_name
  FROM information_schema.columns
 WHERE table_schema = 'public'
   AND table_name   = 'patients'""")

res = cur.fetchall()
print(res)

[('patientid',), ('birthdate',), ('deathdate',), ('ssn',), ('drivers',), ('passport',), ('prefix',), ('firstname',), ('middlename',), ('lastname',), ('suffix',), ('maiden',), ('martial',), ('race',), ('ethinicity',), ('gender',), ('birthplace',), ('address',), ('city',), ('state',), ('county',), ('fips',), ('zip',), ('lat',), ('lon',), ('healthcareexpenses',), ('healthcarecoverage',), ('income',)]


In [25]:
list(map(lambda row: row[0], res))

['patientid',
 'birthdate',
 'deathdate',
 'ssn',
 'drivers',
 'passport',
 'prefix',
 'firstname',
 'middlename',
 'lastname',
 'suffix',
 'maiden',
 'martial',
 'race',
 'ethinicity',
 'gender',
 'birthplace',
 'address',
 'city',
 'state',
 'county',
 'fips',
 'zip',
 'lat',
 'lon',
 'healthcareexpenses',
 'healthcarecoverage',
 'income']